In [1]:
from deap import algorithms, base, creator, tools
import random
from functools import partial
import numpy as np
import scipy
import pandas as pd
import array

In [2]:
class Expression_data:

    def quantilerank(xs):
        ranks = scipy.stats.rankdata(xs, method='average')
        quantile_ranks = [scipy.stats.percentileofscore(ranks, rank, kind='weak') for rank in ranks]
        return np.array(quantile_ranks)

    def __init__(self,expression_data) -> None:
        expression_data["Phylostratum"] = Expression_data.quantilerank(expression_data["Phylostratum"])
        self.full = expression_data
        exps = expression_data.iloc[:, 3:]
        #exps = exps.apply(lambda row: Expression_data.quantilerank(row))
        exps = np.log(exps).apply(lambda x: (x - x.mean()) / x.std(), axis=1)
        self.age_weighted = exps.mul(expression_data["Phylostratum"], axis=0).to_numpy()
        self.expressions_n = exps.to_numpy()
        self.expressions = exps

In [4]:
arr = pd.read_csv("../data/phylo.csv",
                 delimiter=",")
expression_data = Expression_data(arr)
gamma = scipy.stats.gamma(0.4206577838478015, scale=3.4008766040577783*15,loc=0.011999243192420739)


In [35]:
ind_length = expression_data.full.shape[0]
population_size = 400
num_generations = 400
num_parents = round(population_size * 0.15)
init_num_removed = 100 

In [36]:
def create_individual():
    a =  ind_length//(init_num_removed*3)
    b = ind_length//init_num_removed
    individual = array.array("b",random.choices([0,1], weights=(1, random.randint(a,b)), k=ind_length))
    return creator.Individual(individual)

def get_p_value(solution):
    sol = np.array(solution)
    up = sol.dot(expression_data.age_weighted)
    down = sol.dot(expression_data.expressions_n)
    avgs = np.divide(up,down)
    varr = np.var(avgs)
    return 1 - np.array(gamma.cdf(varr))
    
def evaluate_individual(individual):
    num_not_removed = np.sum(individual)
    len_removed = ind_length - num_not_removed
    p_value = get_p_value(individual)

    # Return the fitness values as a tuple
    return len_removed, p_value

In [37]:
creator.create("Fitness", base.Fitness, weights=(-1.0, 1.0))
creator.create("Individual", array.array,typecode='b', fitness=creator.Fitness)

toolbox = base.Toolbox()
toolbox.register("individual", create_individual)
#toolbox.register("population", tools.initRepeat, list, toolbox.individual)
population = [toolbox.individual() for _ in range(population_size)]


/Users/nkalabova/anaconda3/lib/python3.10/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/nkalabova/anaconda3/lib/python3.10/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [38]:
toolbox.register("evaluate", evaluate_individual)
toolbox.register("mate", tools.cxUniform,indpb=0.05)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.003)
toolbox.register("select", tools.selSPEA2)

In [44]:
def custom_stats(individuals):
    generation = 0  # Variable to keep track of the generation
    def func(population):
        nonlocal generation
        if generation % 10 == 0:
            # Calculate and print the desired statistics for each objective
            objectives = len(population[0].fitness.values)
            for obj in range(objectives):
                values = [ind.fitness.values[obj] for ind in population]
                
                print(f"Generation {generation}: Objective {obj+1} - Max Value = {values[0]}")
        generation += 1
    return func

In [45]:
stats = tools.Statistics()
"""
stats.register("Num removed", lambda x: np.max([ind.fitness.values[0] for ind in x]))
stats.register("P-value", lambda x: np.max([ind.fitness.values[1] for ind in x]))
"""
stats.register("custom", custom_stats(population))

In [46]:
population, logbook = algorithms.eaMuPlusLambda(population,toolbox, mu=num_parents, lambda_ = population_size,cxpb=0.25, mutpb=0.5, ngen=num_generations,stats=stats, verbose=True)

gen	nevals	custom                                                 
0  	0     	<function custom_stats.<locals>.func at 0x7f8844960040>
1  	316   	<function custom_stats.<locals>.func at 0x7f88449b77f0>
2  	297   	<function custom_stats.<locals>.func at 0x7f88449b7d00>


KeyboardInterrupt: 

In [ ]:
pareto_front = tools.sortNondominated(population, k=population_size,first_front_only=True)
par = np.array([list(x) for x in pareto_front[0]])
np.savetxt("../results/best_solutions.csv", par, delimiter=",")